In [2]:
import os
import shutil
import numpy as np

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, GRU, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.model_selection import ParameterGrid

## Parameters

In [3]:
VOCABULARY_SIZE = 2500
EMBEDDING_DIMENSIONS = 8
SEQUENCE_LENGTH = 16
EPOCHS = 30

In [4]:
# Define generator model
def make_generator_model(cfg):
    reg = l2(cfg["regularizer"]) if cfg["regularizer"] is not None else None

    model = Sequential()

    model.add(Embedding(VOCABULARY_SIZE, EMBEDDING_DIMENSIONS, input_length=SEQUENCE_LENGTH))
    model.add(GRU(cfg["layers"][0], return_sequences=True, kernel_initializer=cfg['kernel_init'], kernel_regularizer=reg))

    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(GRU(cfg["layers"][1], kernel_regularizer=reg))

    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(VOCABULARY_SIZE, activation="softmax"))
    return model

In [5]:
# Training loop
def train(model, X_train, y_train, X_valid, y_valid, batch_size, learning_rate, epochs):  
    optimizer = Adam(learning_rate=learning_rate, amsgrad=True)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

    checkpoint_dir = os.path.sep.join(['model', 'tmp'])
    checkpoint_filepath = os.path.sep.join([checkpoint_dir, 'checkpoint'])

    cbs_list = [
        ModelCheckpoint(checkpoint_filepath, monitor='val_accuracy', mode='max', save_weights_only=True, verbose=0)
    ]

    history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=epochs, batch_size=batch_size, callbacks=cbs_list)

    model.load_weights(checkpoint_filepath)
    shutil.rmtree(checkpoint_dir)

    return history

In [6]:
# Plot model performance
def plot_to_pdf(history, i, pdf):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.plot(history.history["accuracy"])
    ax1.plot(history.history["val_accuracy"])
    ax1.set_title("model accuracy")
    ax1.set_ylabel("accuracy")
    ax1.set_xlabel("epoch")
    ax1.legend(["train", "val"], loc="upper left")

    ax2.plot(history.history["loss"])
    ax2.plot(history.history["val_loss"])
    ax2.set_title("model loss")
    ax2.set_ylabel("loss")
    ax2.set_xlabel("epoch")
    ax2.legend(["train", "val"], loc="upper left")

    plt.suptitle(f"cfg n: {i}")
    plt.tight_layout()

    pdf.savefig(fig)
    plt.close(fig)

In [7]:
def evaluate_model(model, X_test_eval, y_test_eval):
    loss, acc = model.evaluate(X_test_eval, y_test_eval)
    print(loss, acc)
    return acc

In [8]:
# Grid search
def grid_search(cfgs, X_train, y_train, X_valid, y_valid, X_test_eval, y_test_eval, pdf):
    scores = []

    for i, cfg in cfgs.items():
        print(f"\nCurrent config: {i}")
        model = make_generator_model(cfg)
        history = train(model, X_train, y_train, X_valid, y_valid, cfg["batch_size"], cfg["learning_rate"], EPOCHS)
        plot_to_pdf(history, i, pdf)
        accuracy = evaluate_model(model, X_test_eval, y_test_eval)
        scores.append((i, accuracy, model))
        
        # How many models to keep/track in list
        if len(scores) > 3:
            scores.sort(key=lambda tup: tup[1], reverse=True)
            del scores[-1]

    pdf.close()

    return scores

In [10]:
# Configs
param_grid = {
    "learning_rate": [0.0004],
    "batch_size": [256, 512],
    "regularizer": [0.001, None],
    "layers": [[32, 64], [64, 128]],
    "kernel_init": ['he_uniform', 'glorot_uniform']
}

cfgs = {i: cfg for i, cfg in enumerate(list(ParameterGrid(param_grid)), 1)}
for key, value in cfgs.items():
    print(key, value)

output_path = "output"

if not os.path.exists(output_path):
    os.mkdir(output_path)

pdf = PdfPages(os.path.join(output_path, "grid_search_graphs"))

1 {'batch_size': 256, 'kernel_init': 'he_uniform', 'layers': [32, 64], 'learning_rate': 0.0004, 'regularizer': 0.001}
2 {'batch_size': 256, 'kernel_init': 'he_uniform', 'layers': [32, 64], 'learning_rate': 0.0004, 'regularizer': None}
3 {'batch_size': 256, 'kernel_init': 'he_uniform', 'layers': [64, 128], 'learning_rate': 0.0004, 'regularizer': 0.001}
4 {'batch_size': 256, 'kernel_init': 'he_uniform', 'layers': [64, 128], 'learning_rate': 0.0004, 'regularizer': None}
5 {'batch_size': 256, 'kernel_init': 'glorot_uniform', 'layers': [32, 64], 'learning_rate': 0.0004, 'regularizer': 0.001}
6 {'batch_size': 256, 'kernel_init': 'glorot_uniform', 'layers': [32, 64], 'learning_rate': 0.0004, 'regularizer': None}
7 {'batch_size': 256, 'kernel_init': 'glorot_uniform', 'layers': [64, 128], 'learning_rate': 0.0004, 'regularizer': 0.001}
8 {'batch_size': 256, 'kernel_init': 'glorot_uniform', 'layers': [64, 128], 'learning_rate': 0.0004, 'regularizer': None}
9 {'batch_size': 512, 'kernel_init': 'he

In [11]:
X_train = np.load(os.path.sep.join(['data', 'train_sequences.npy']))
y_train = np.load(os.path.sep.join(['data', 'train_labels.npy']))

X_valid = np.load(os.path.sep.join(['data', 'valid_sequences.npy']))
y_valid = np.load(os.path.sep.join(['data', 'valid_labels.npy']))

X_test_eval = np.load(os.path.sep.join(['data', 'eval_sequences.npy']))
y_test_eval = np.load(os.path.sep.join(['data', 'eval_labels.npy']))

In [12]:
# Run grid search
scores = grid_search(cfgs, X_train, y_train, X_valid, y_valid, X_test_eval, y_test_eval, pdf)


Current config: 1
Epoch 1/30
838/838 [==============================] - 15s 9ms/step - loss: 5.9158 - accuracy: 0.1772 - val_loss: 4.5043 - val_accuracy: 0.2403
Epoch 2/30
838/838 [==============================] - 7s 8ms/step - loss: 4.3804 - accuracy: 0.2386 - val_loss: 4.1438 - val_accuracy: 0.2627
Epoch 3/30
838/838 [==============================] - 7s 8ms/step - loss: 4.1679 - accuracy: 0.2530 - val_loss: 4.0199 - val_accuracy: 0.2741
Epoch 4/30
838/838 [==============================] - 7s 8ms/step - loss: 4.0608 - accuracy: 0.2608 - val_loss: 3.9536 - val_accuracy: 0.2792
Epoch 5/30
838/838 [==============================] - 7s 8ms/step - loss: 3.9917 - accuracy: 0.2653 - val_loss: 3.9083 - val_accuracy: 0.2829
Epoch 6/30
838/838 [==============================] - 7s 8ms/step - loss: 3.9392 - accuracy: 0.2696 - val_loss: 3.8745 - val_accuracy: 0.2863
Epoch 7/30
838/838 [==============================] - 7s 8ms/step - loss: 3.8987 - accuracy: 0.2733 - val_loss: 3.8508 - val_acc

In [13]:
# Print scores
for i, accuracy, model in scores:
    print("cfg:", i, "acc:", accuracy, "model:", model)

cfg: 8 acc: 0.3056475520133972 model: <keras.engine.sequential.Sequential object at 0x7f693641fa90>
cfg: 4 acc: 0.3055626451969147 model: <keras.engine.sequential.Sequential object at 0x7f694bd22490>
cfg: 12 acc: 0.30411890149116516 model: <keras.engine.sequential.Sequential object at 0x7f69369c9cd0>


In [14]:
# Save top model
scores[0][2].save(f'saved_model_cfg_{scores[0][0]}')

INFO:tensorflow:Assets written to: saved_model_cfg_8/assets


INFO:tensorflow:Assets written to: saved_model_cfg_8/assets
